##### Copyright 2020 The OpenFermion Developers

# Introduction to FQE

OpenFermion-FQE is an emulator for quantum computing specialized for simulations of Fermionic many-body problems, where FQE stands for 'Fermionic quantum emulator.' By focusing on Fermionic physics, OpenFermion-FQE realizes far more efficient emulation of quantum computing than generic quantum emulators such as Cirq, both in computation and memory costs; the speed-up and improved memory footprint originate from the use of the spin and number symmetries as well as specialized and highly optimized algorithms.

The examples of the problems that can be simulated by OpenFermion-FQE include those in molecular electronic structure, condensed matter physics, and nuclear physics.

The initial version of OpenFermion-FQE has been developed in collaboration between [QSimulate](https://qsimulate.com)  and Google Quantum AI. The source code is found in the GitHub repository (https://github.com/quantumlib/OpenFermion-FQE).

This tutorial will describe the data structures and conventions of the library.

In [1]:
import fqe
import numpy as np

## The FQE Wavefunction

The `Wavefunction` is an interface to the objects that hold the actual wavefunction data.  As mentioned, the wavefunction is partitioned into sectors with fixed particle and $Sz$ quantum number.  This partitioning information is the necessary information for initializing a `Wavefunction` object.  

As an example, we consider initializing a wavefunction with four spatial orbitals, four electrons, and different $Sz$ expectation values.  The `Wavefunction` object takes a list of these sectors `[[n_electrons, sz, n_orbits]]`.

In [2]:
wfn = fqe.Wavefunction([[4, 4, 4], [4, 2, 4], [4, 0, 4], [4, -2, 4], [4, -4, 4]])

This command initializes a wavefunction with the following block structure:

![Image of wf sectors](./wf_init_sectors.png)

Each sector corresponds to a set of bit strings 

$$
\vert I \rangle = \vert I_{\alpha}I_{\beta}\rangle
$$ 

that encode a fixed particle number and fixed $Sz$ expectation. The coefficients associated with the bitstrings in these sectors are formed into matrices.  This helps with efficient vectorized computations. The row-index of the array corresponds to the $\alpha$ spin-orbital number occupation index and the column-index corresponds to the $\beta$-strings.  The `Wavefunction` object provides tools to access sectors or perform basic mathematical operations on this *vector*.  

## Methods to initialize wavefunctions

FQE wavefunctions can be initialized by calling the constructor directly.

In [3]:
wfn_fqe = fqe.Wavefunction([[2, -2, 4]], broken=None)

When wavefunctions are first created, they are initialized to empty values. We can see this by printing out the wavefunction. 

In [4]:
wfn_fqe.print_wfn()

Sector N = 2 : S_z = -2


To set the values of a wavefunction, we can use the `set_wfn` method with a provided `strategy`.

In [5]:
wfn_fqe.set_wfn(strategy="ones")
wfn_fqe.print_wfn()

Sector N = 2 : S_z = -2
a'0000'b'0011' (1+0j)
a'0000'b'0101' (1+0j)
a'0000'b'1001' (1+0j)
a'0000'b'0110' (1+0j)
a'0000'b'1010' (1+0j)
a'0000'b'1100' (1+0j)


Users can access the wavefunction through the `get_sector` method.  This returns the entire matrix of data representing the specified sector of the wavefunction.  For example, we can grab the sector corresponding to $n = 2$ and $sz = -2$ by doing the following.

In [6]:
interesting_states = wfn_fqe.get_coeff((2, -2))
print(interesting_states)

[[1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j]]


Other than the `Wavefunction` constructor, several utility methods are available to initialize wavefunctions. The function `fqe.get_wavefunction` builds a wavefunction with definite particle number and spin.

In [7]:
wfn_fqe = fqe.get_wavefunction(4, -2, 10)

The function `fqe.get_wavefunction_multiple` constructs multiple wavefunctions with different particle number, spin, and orbital number.

In [8]:
wfn_fqe1, wfn_fqe2 = fqe.get_wavefunction_multiple([[4, 0, 10], [5, -5, 10]])

There are also functions like `fqe.get_number_conserving_wavefunction` and  `fqe.get_spin_conserving_wavefunction` to get number or spin conserving wavefunctions, respectively.

In [9]:
# Get a spin conserving wavefunction.
spin_conserving_wfn = fqe.get_spin_conserving_wavefunction(2, 4)

# Get a number conserving wavefunction.
number_conserving_wfn = fqe.get_number_conserving_wavefunction(2, 4)

### Conversions between FQE and Cirq wavefunction representations

Wavefunctions on $n$ qubits in Cirq are represented by Numpy arrays with $2^n$ amplitudes.

In [10]:
nqubits = 4

cirq_wfn = np.random.rand(2**nqubits) + 1.0j * np.random.rand(2**nqubits)
cirq_wfn /= np.linalg.norm(cirq_wfn)

print("Cirq wavefunction:")
print(*cirq_wfn, sep="\n")

Cirq wavefunction:
(0.10201509862505637+0.12675829526963237j)
(0.15969891362740796+0.024958211027290744j)
(0.2543768640377393+0.11305711141940487j)
(0.24426533648563178+0.14936038515969494j)
(0.233008441765372+0.20105362253776218j)
(0.26827015872837584+0.08689928853505362j)
(0.2287940883949938+0.07657203676252496j)
(0.19978463063814364+0.1542276101518718j)
(0.11580074335500472+0.011348543996133758j)
(0.07324095131005602+0.15297636722961228j)
(0.26094884799827184+0.018808790628639133j)
(0.2384229997806729+0.03840009823382135j)
(0.20796091638437797+0.20300323644429077j)
(0.21629203583187187+0.2551914840818459j)
(0.03368777235501211+0.021094179583674138j)
(0.29122106788247193+0.19990944727160503j)


To convert from this representation to the FQE representation, the function `fqe.from_cirq` can be used.

In [11]:
fqe_wfn = fqe.from_cirq(cirq_wfn, thresh=0.0001)
fqe_wfn.print_wfn()

Sector N = 0 : S_z = 0
a'00'b'00' (0.10201509862505637+0.12675829526963237j)
Sector N = 1 : S_z = -1
a'00'b'01' (0.233008441765372+0.20105362253776218j)
a'00'b'10' (0.15969891362740796+0.024958211027290744j)
Sector N = 1 : S_z = 1
a'01'b'00' (0.11580074335500472+0.011348543996133758j)
a'10'b'00' (0.2543768640377393+0.11305711141940487j)
Sector N = 2 : S_z = -2
a'00'b'11' (0.26827015872837584+0.08689928853505362j)
Sector N = 2 : S_z = 0
a'01'b'01' (0.20796091638437797+0.20300323644429077j)
a'01'b'10' (0.07324095131005602+0.15297636722961228j)
a'10'b'01' (-0.2287940883949938-0.07657203676252496j)
a'10'b'10' (0.24426533648563178+0.14936038515969494j)
Sector N = 2 : S_z = 2
a'11'b'00' (0.26094884799827184+0.018808790628639133j)
Sector N = 3 : S_z = -1
a'01'b'11' (0.21629203583187187+0.2551914840818459j)
a'10'b'11' (-0.19978463063814364-0.1542276101518718j)
Sector N = 3 : S_z = 1
a'11'b'01' (-0.03368777235501211-0.021094179583674134j)
a'11'b'10' (0.23842299978067288+0.03840009823382135j)
Se

> *Note*: The `thresh` argument is the value below which amplitudes are considered zero.

We can convert back to the Cirq representation using `fqe._to_cirq`.

In [12]:
cirq_wfn_from_fqe = fqe.to_cirq(fqe_wfn)

print("Cirq wavefunction from FQE:")
print(*cirq_wfn_from_fqe, sep="\n")

Cirq wavefunction from FQE:
(0.10201509862505637+0.12675829526963237j)
(0.15969891362740796+0.024958211027290744j)
(0.2543768640377393+0.11305711141940487j)
(0.24426533648563178+0.14936038515969494j)
(0.233008441765372+0.20105362253776218j)
(0.26827015872837584+0.08689928853505362j)
(0.2287940883949938+0.07657203676252496j)
(0.19978463063814364+0.1542276101518718j)
(0.11580074335500472+0.011348543996133758j)
(0.07324095131005602+0.15297636722961228j)
(0.26094884799827184+0.018808790628639133j)
(0.23842299978067286+0.03840009823382135j)
(0.20796091638437797+0.20300323644429077j)
(0.21629203583187187+0.2551914840818459j)
(0.03368777235501211+0.02109417958367413j)
(0.29122106788247193+0.199909447271605j)


An important thing to note in these conversions is the ordering of the $\alpha$ and $\beta$ strings in the converted wavefunctions.  The FQE uses the OpenFermion convention of interleaved $\alpha$ and $\beta$ orbitals.  Thus when converting to Cirq we first convert each bitstring into an OpenFermion operator and then call normal ordering.

### Printing and saving wavefunctions

Printing is currently available as alpha beta strings followed by the coefficient as well as orbital occupation representation.

In [13]:
print('String formatting')
fqe_wfn.print_wfn(fmt='str')

String formatting
Sector N = 0 : S_z = 0
a'00'b'00' (0.10201509862505637+0.12675829526963237j)
Sector N = 1 : S_z = -1
a'00'b'01' (0.233008441765372+0.20105362253776218j)
a'00'b'10' (0.15969891362740796+0.024958211027290744j)
Sector N = 1 : S_z = 1
a'01'b'00' (0.11580074335500472+0.011348543996133758j)
a'10'b'00' (0.2543768640377393+0.11305711141940487j)
Sector N = 2 : S_z = -2
a'00'b'11' (0.26827015872837584+0.08689928853505362j)
Sector N = 2 : S_z = 0
a'01'b'01' (0.20796091638437797+0.20300323644429077j)
a'01'b'10' (0.07324095131005602+0.15297636722961228j)
a'10'b'01' (-0.2287940883949938-0.07657203676252496j)
a'10'b'10' (0.24426533648563178+0.14936038515969494j)
Sector N = 2 : S_z = 2
a'11'b'00' (0.26094884799827184+0.018808790628639133j)
Sector N = 3 : S_z = -1
a'01'b'11' (0.21629203583187187+0.2551914840818459j)
a'10'b'11' (-0.19978463063814364-0.1542276101518718j)
Sector N = 3 : S_z = 1
a'11'b'01' (-0.03368777235501211-0.021094179583674134j)
a'11'b'10' (0.23842299978067288+0.0384

In [14]:
print('Occupation formatting')
fqe_wfn.print_wfn(fmt='occ')

Occupation formatting
Sector N = 0 : S_z = 0
.. (0.10201509862505637+0.12675829526963237j)
Sector N = 1 : S_z = -1
.b (0.233008441765372+0.20105362253776218j)
b. (0.15969891362740796+0.024958211027290744j)
Sector N = 1 : S_z = 1
.a (0.11580074335500472+0.011348543996133758j)
a. (0.2543768640377393+0.11305711141940487j)
Sector N = 2 : S_z = -2
bb (0.26827015872837584+0.08689928853505362j)
Sector N = 2 : S_z = 0
.2 (0.20796091638437797+0.20300323644429077j)
ba (0.07324095131005602+0.15297636722961228j)
ab (-0.2287940883949938-0.07657203676252496j)
2. (0.24426533648563178+0.14936038515969494j)
Sector N = 2 : S_z = 2
aa (0.26094884799827184+0.018808790628639133j)
Sector N = 3 : S_z = -1
b2 (0.21629203583187187+0.2551914840818459j)
2b (-0.19978463063814364-0.1542276101518718j)
Sector N = 3 : S_z = 1
a2 (-0.03368777235501211-0.021094179583674134j)
2a (0.23842299978067288+0.03840009823382135j)
Sector N = 4 : S_z = 0
22 (-0.29122106788247193-0.199909447271605j)


Wavefunctions can also be saved to disk using the `save` method which takes a filename and optional path.

# Action on Wavefunctions: Fermionic algebra operations and their unitaries on the state

FermionOperators can be directly passed in to create a new wavefunction based on application of the operators.  The FermionOperators are parsed according to the interleaved $\alpha$ $\beta$ indexing of the spin-orbitals.  This means that odd index FermionOperators correspond to $\beta$-spin orbitals and even are $\alpha$-spin orbitals.  

Sharp Edge:

The user must be careful to not break the symmetry of the wavefunction.  If a request to apply an operator to a state takes the wavefunction outside of the specified symmetry sector the FQE will not execute the command.  Effectively, the FQE requires the user to have more knowledge of what type of operations their `Wavefunction` object can support.

In [15]:
from openfermion import FermionOperator, hermitian_conjugated

ops = FermionOperator('2^ 0', 1.2)

new_wfn = fqe_wfn.apply(ops + hermitian_conjugated(ops))

## Unitary operations

Any simulator backend must be able to perform unitary evolution on a state.  The FQE accomplishes this by implementing code for evolving a state via the action of a unitary generated by fermionic generators.  Given a fermion operator $g$, the unitary 

$$
e^{-i (g + g^{\dagger})}
$$

can be applied to the state.  It can be shown that this evolution operator can be rewritten as 

$$
e^{-i(g + g^{\dagger})\epsilon } = \cos\left(\epsilon\right) \mathbb{I}_{s}(gg^{\dagger}) +  \cos\left(\epsilon\right) \mathbb{I}_{s}(g^{\dagger}g) - i\sin\left(\epsilon\right) \left(g + g^{\dagger}\right) \left[\mathbb{I}_{s}(gg^{\dagger}) + \mathbb{I}_{s}(g^{\dagger}g)\right] + \mathbb{I}_{!s}
$$

The $\mathbb{I}_{!s}$ is for setting the coefficients of the unitary that are not in the subspace $\mathcal{H}_{s} \subset \mathcal{H}$ where $gg^{\dagger}$ is 0.   

The user can specify a fermionic monomial in OpenFermion and use the `time_evolve` method of the `Wavefunction` object to call the evolution. All the rules for preserving symmetries must be maintained as before.

In [16]:
i, j, theta = 0, 1, np.pi / 3
op = (FermionOperator(((2 * i, 1), (2 * j, 0)), coefficient=-1j * theta) + 
      FermionOperator(((2 * j, 1), (2 * i, 0)), coefficient=1j * theta))

new_wfn = fqe_wfn.time_evolve(1.0, op)

new_wfn.print_wfn()

Sector N = 0 : S_z = 0
a'00'b'00' (0.10201509862505637+0.12675829526963237j)
Sector N = 1 : S_z = -1
a'00'b'01' (0.233008441765372+0.20105362253776218j)
a'00'b'10' (0.15969891362740796+0.024958211027290744j)
Sector N = 1 : S_z = 1
a'01'b'00' (-0.16239645471420003-0.09223605856962551j)
a'10'b'00' (0.22747481754142573+0.06635668310631966j)
Sector N = 2 : S_z = -2
a'00'b'11' (0.26827015872837584+0.08689928853505362j)
Sector N = 2 : S_z = 0
a'01'b'01' (0.30212195097795613+0.16781494727800797j)
a'01'b'10' (-0.17491951100548303-0.05286170425251792j)
a'10'b'01' (0.06570239238566586+0.13751994142995227j)
a'10'b'10' (0.18556119267466362+0.207161612779349j)
Sector N = 2 : S_z = 2
a'11'b'00' (0.26094884799827184+0.018808790628639133j)
Sector N = 3 : S_z = -1
a'01'b'11' (0.2811645833342592+0.2611607703974067j)
a'10'b'11' (0.08742208234758324+0.14388850296839478j)
Sector N = 3 : S_z = 1
a'11'b'01' (-0.03368777235501211-0.021094179583674134j)
a'11'b'10' (0.23842299978067288+0.03840009823382135j)
Sec

In other tutorials we will do a deeper dive into supported time-evolution operations.  To serve a full functioning time-evolution platform the FQE also implements arbitrary time-evolution of full Hamiltonian operators consisting of sums of non-commuting terms.  The Taylor and Chebyshev expansion methods are used to do the exact time evolution.  